In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 25616894
paper_name = 'junne_hoepfner_2015' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
sheet_names = ['Decatransin Cpd 1 Exp 1','Decatransin Cpd 2 Exp 2','Cotransin Cpd 2','Cotansin Cpd 3 (HUN-7293)']

In [7]:
original_data1_list = []
original_data2_list = []
for s in sheet_names:
    original_data = pd.read_excel('raw_data/Junnet_et_al_HIPHOPrawdata.xlsx', sheet_name=s)
    print('Original data dimensions: %d x %d' % (original_data.shape))
    original_data['orf'] = original_data['SYSTEMATIC_NAME'].astype(str)
    original_data['orf'] = clean_orf(original_data['orf'])
    original_data['orf'] = translate_sc(original_data['orf'], to='orf')
    
    # Make sure everything translated ok
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    
    original_data['data'] = original_data['Z_SCORE']
    original_data.set_index('orf', inplace=True)
    
    original_data1 = original_data.loc[original_data['EXPERIMENT_TYPE']=='HIP',:].copy()
    original_data2 = original_data.loc[original_data['EXPERIMENT_TYPE']=='HOP',:].copy()
    
    original_data1 = original_data1[['data']].copy()
    original_data2 = original_data2[['data']].copy()
    
    original_data1 = original_data1.groupby(original_data1.index).mean()
    original_data2 = original_data2.groupby(original_data2.index).mean()
    
    print(original_data1.shape)
    print(original_data2.shape)
    
    original_data1_list.append(original_data1)
    original_data2_list.append(original_data2)

Original data dimensions: 10122 x 7
Empty DataFrame
Columns: [EXPERIMENT_TYPE, GENE_NAME, SENSITIVITY SCORE, SCORE_TYPE, SYSTEMATIC_NAME, VIABILITY, Z_SCORE, orf]
Index: []
(5706, 1)
(4400, 1)
Original data dimensions: 10270 x 7
Empty DataFrame
Columns: [EXPERIMENT_TYPE, GENE_NAME, SENSITIVITY SCORE, SCORE_TYPE, SYSTEMATIC_NAME, VIABILITY, Z_SCORE, orf]
Index: []
(5755, 1)
(4499, 1)
Original data dimensions: 10254 x 7
Empty DataFrame
Columns: [EXPERIMENT_TYPE, GENE_NAME, SENSITIVITY SCORE, SCORE_TYPE, SYSTEMATIC_NAME, VIABILITY, Z_SCORE, orf]
Index: []
(5756, 1)
(4482, 1)
Original data dimensions: 10256 x 7
Empty DataFrame
Columns: [EXPERIMENT_TYPE, GENE_NAME, SENSITIVITY SCORE, SCORE_TYPE, SYSTEMATIC_NAME, VIABILITY, Z_SCORE, orf]
Index: []
(5744, 1)
(4496, 1)


In [17]:
original_data1 = pd.concat(original_data1_list, axis=1)
original_data2 = pd.concat(original_data2_list, axis=1)

In [18]:
original_data1.columns = np.arange(4)
original_data1[4] = original_data1[[0,1]].mean(axis=1)
original_data1.drop(columns=[0,1], inplace=True)
original_data1.head()

In [20]:
original_data2.columns = np.arange(4)
original_data2[4] = original_data2[[0,1]].mean(axis=1)
original_data2.drop(columns=[0,1], inplace=True)
original_data2.head()

,2,3,4
YAL001C,-0.077701,-2.199026,-2.274844
YAL002W,-0.076000,0.610906,-3.036789
YAL004W,-0.014744,-1.001728,-0.188456
YAL005C,0.310500,-1.278865,-0.020341
YAL007C,1.624255,0.133624,-0.715401


In [21]:
original_data = original_data1.join(original_data2, lsuffix='_1', rsuffix='_2')

In [24]:
original_data.index.name='orf'

In [25]:
original_data.head()

,2_1,3_1,4_1,2_2,3_2,4_2
orf,,,,,,
YAL001C,0.026759,-0.574845,0.076030,-0.077701,-2.199026,-2.274844
YAL002W,0.053244,-1.485876,0.006303,-0.076000,0.610906,-3.036789
YAL003W,0.080687,-0.483062,0.286196,NaN,NaN,NaN
YAL004W,-0.369822,-1.214900,0.221975,-0.014744,-1.001728,-0.188456
YAL005C,0.171241,0.224126,0.269686,0.310500,-1.278865,-0.020341


# Prepare the final dataset

In [27]:
data = original_data.copy()

In [28]:
dataset_ids = [5265,5267,773,5264,5266,772]
datasets = datasets.reindex(index=dataset_ids)

In [29]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [30]:
data.head()

dataset_id,5265,5267,773,5264,5266,772
data_type,value,value,value,value,value,value
orf,,,,,,
YAL001C,0.026759,-0.574845,0.076030,-0.077701,-2.199026,-2.274844
YAL002W,0.053244,-1.485876,0.006303,-0.076000,0.610906,-3.036789
YAL003W,0.080687,-0.483062,0.286196,NaN,NaN,NaN
YAL004W,-0.369822,-1.214900,0.221975,-0.014744,-1.001728,-0.188456
YAL005C,0.171241,0.224126,0.269686,0.310500,-1.278865,-0.020341


## Subset to the genes currently in SGD

In [31]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 10


In [32]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,5265,5267,773,5264,5266,772
,data_type,value,value,value,value,value,value
gene_id,orf,,,,,,
1,YAL001C,0.026759,-0.574845,0.076030,-0.077701,-2.199026,-2.274844
2,YAL002W,0.053244,-1.485876,0.006303,-0.076000,0.610906,-3.036789
3,YAL003W,0.080687,-0.483062,0.286196,NaN,NaN,NaN
1863,YAL004W,-0.369822,-1.214900,0.221975,-0.014744,-1.001728,-0.188456
4,YAL005C,0.171241,0.224126,0.269686,0.310500,-1.278865,-0.020341


# Normalize

In [33]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [34]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [35]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,5265,5267,773,5264,5266,772,5265,5267,773,5264,5266,772
,data_type,value,value,value,value,value,value,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,
1,YAL001C,0.026759,-0.574845,0.076030,-0.077701,-2.199026,-2.274844,0.134713,-0.758243,-0.115098,-0.124187,-2.218241,-1.930218
2,YAL002W,0.053244,-1.485876,0.006303,-0.076000,0.610906,-3.036789,0.152598,-1.685508,-0.174066,-0.122939,0.485040,-2.606917
3,YAL003W,0.080687,-0.483062,0.286196,NaN,NaN,NaN,0.171131,-0.664825,0.062639,NaN,NaN,NaN
1863,YAL004W,-0.369822,-1.214900,0.221975,-0.014744,-1.001728,-0.188456,-0.133113,-1.409703,0.008327,-0.078005,-1.066387,-0.077253
4,YAL005C,0.171241,0.224126,0.269686,0.310500,-1.278865,-0.020341,0.232286,0.054965,0.048676,0.160574,-1.333005,0.072053


# Print out

In [36]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [37]:
from IO.save_data_to_db3 import *

In [38]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/6 [00:00<?, ?it/s]

Deleting all datasets for PMID 25616894...
Inserting the new data...


100%|██████████| 6/6 [00:54<00:00,  9.04s/it]

Updating the data_modified_on field...
